# Module 0.0: Qubit States

## Section 0.0.1: Dirac Notation Basics

Quantum states are represented using **Dirac notation**:
- |0⟩ = [1, 0]ᵗ  (certain measurement of 0)
- |1⟩ = [0, 1]ᵗ  (certain measurement of 1)
- |+⟩ = (1/√2)(|0⟩ + |1⟩)
- |−⟩ = (1/√2)(|0⟩ − |1⟩)

These are **unit vectors** in a 2-dimensional complex space.

### Examples

In [1]:
from qiskit.quantum_info import Statevector
import rustworkx as retworkx

print(Statevector([1, 0]))  # |0⟩
print(Statevector([0, 1]))  # |1⟩
print(Statevector([1/2**0.5, 1/2**0.5]))  # |+⟩
print(Statevector([1/2**0.5, -1/2**0.5]))  # |−⟩

Statevector([1.+0.j, 0.+0.j],
            dims=(2,))
Statevector([0.+0.j, 1.+0.j],
            dims=(2,))
Statevector([0.70710678+0.j, 0.70710678+0.j],
            dims=(2,))
Statevector([ 0.70710678+0.j, -0.70710678+0.j],
            dims=(2,))


## Section 0.0.2: Probabilities from Amplitudes

If a state is |ψ⟩ = α|0⟩ + β|1⟩, then:
- P(0) = |α|²
- P(1) = |β|²

The vector represents the **probabilities** of outcomes upon measurement.

In [2]:
sv = Statevector([1/2**0.5, 1/2**0.5])  # |+⟩ state
print(sv.probabilities_dict())

{np.str_('0'): np.float64(0.4999999999999999), np.str_('1'): np.float64(0.4999999999999999)}


## Section 0.0.3: Normalization

A quantum state must be **normalized**:

|α₀|² + |α₁|² + ... + |αₙ|² = 1

This ensures that the total measurement probability is 1.

In [3]:
from math import sqrt
sv_valid = Statevector([sqrt(3)/2, 1/2])
print(sv_valid.probabilities())

# Invalid (unnormalized) example:
# sv_invalid = Statevector([2, 2])  # This will raise an error

[0.75 0.25]


# 0.0.4 Global Phase (and why it usually doesn’t matter)

**Idea.** Multiplying a quantum state by a complex phase $e^{i\theta}$ does **not** change any measurement probabilities.  
For a pure state $|\psi\rangle$, the globally phased state $e^{i\theta}|\psi\rangle$ yields the **same statistics** in every basis:
$$
P(x)=|\langle x|\psi\rangle|^2 = |\langle x|e^{i\theta}\psi\rangle|^2.
$$

- **Global phase**: one phase factor multiplying the **entire** state (or the entire unitary). Physically unobservable.  
- **Relative phase**: phase **differences** between components (e.g., $|0\rangle + e^{i\phi}|1\rangle$). This **is** observable via interference.

### Why probabilities don’t change
Given amplitudes $\alpha_x = \langle x|\psi\rangle$, the phased amplitudes are $\alpha'_x = e^{i\theta}\alpha_x$, so
$|\alpha'_x|^2 = |e^{i\theta}|^2 |\alpha_x|^2 = |\alpha_x|^2$.

### When phase *does* matter
- If a phase is applied to **only part** of a superposition (e.g., a $Z$ rotation on $|1\rangle$ but not $|0\rangle$), that’s a **relative** phase and it changes interference patterns.
- A gate or circuit can differ by a **global phase** and still represent the **same physical operation**. Many compilers recognize “equivalence up to global phase.”

> **Exam tip:** If two states or two unitaries differ only by a factor $e^{i\theta}$, they are physically equivalent for all measurement outcomes.

### Qiskit note (v2.x)
- Circuits carry an optional `global_phase` (e.g., `QuantumCircuit(..., global_phase=theta)`).  
- Many equivalence checks in Qiskit treat circuits/operators as **equal up to global phase** when appropriate.

#### Examples
- $|+\rangle = \tfrac{1}{\sqrt{2}}(|0\rangle + |1\rangle)$ and $e^{i\pi/7}|+\rangle$ are indistinguishable by measurement.  
- The matrices $H$ and $-H$ (Hadamard and negative Hadamard) differ by a global phase $\pi$; they implement the **same** physical transformation.

**Key takeaways**
1. Global phase is unobservable; relative phase is observable.  
2. Compilers/simulators often ignore global phase differences when checking equivalence.  
3. Be careful to distinguish a single, overall $e^{i\theta}$ from state-component-specific phases.


### Example: T gate — global vs relative phase

**Question.** Consider the matrix $T=\mathrm{diag}(1,e^{i\pi/4})$ (we'll find out later this is the T gate). When applied to a qubit in the pure state $|1\rangle$, what phase change does it introduce?

A. A **relative** phase of $\pi/4$ between $|0\rangle$ and $|1\rangle$.  
B. A **global** phase of $e^{i\pi/4}$ on the state.  
C. A **global** phase of $e^{i\pi/8}$ on the state.  
D. No phase change at all.

---

**Answer:** **B**

**Explanation:**  
$T|1\rangle = e^{i\pi/4}|1\rangle$. Since the state has no $|0\rangle$ component, the factor $e^{i\pi/4}$ multiplies the entire state—this is a **global** phase (physically unobservable). A **relative** phase appears only for superpositions such as $|+\rangle = (|0\rangle+|1\rangle)/\sqrt{2}$, where $T$ yields $(|0\rangle + e^{i\pi/4}|1\rangle)/\sqrt{2}$. Note: $T = e^{i\pi/8} R_z(\pi/4)$; the $e^{i\pi/8}$ relates $T$ to $R_z$ and is not the phase acquired by $|1\rangle$.


#### Verify with Qiskit

In [1]:
# Demonstration: global phase does not change probabilities (Qiskit v2.x)
import numpy as np
from qiskit.quantum_info import Statevector
from qiskit import QuantumCircuit

theta = 0.37  # some arbitrary global phase

# Prepare |+> = (|0> + |1>)/sqrt(2)
qc = QuantumCircuit(1)
qc.h(0)
psi = Statevector.from_instruction(qc)

# Apply a global phase by multiplying the statevector data
psi_phase = np.exp(1j*theta) * psi.data

# Compare measurement probabilities
probs_psi = np.abs(psi.data)**2
probs_phase = np.abs(psi_phase)**2

print("Probabilities equal:", np.allclose(probs_psi, probs_phase))
print("Probs:", probs_psi)

# Optional: show that two circuits differing by global phase act the same up to a phase
qc1 = QuantumCircuit(1)
qc1.h(0)

qc2 = QuantumCircuit(1, global_phase=theta)
qc2.h(0)

# Apply both to |0> and compare the resulting states up to a global phase
sv1 = Statevector.from_instruction(qc1)
sv2 = Statevector.from_instruction(qc2)

# Compute the phase that aligns sv2 to sv1
phase_est = np.angle(np.vdot(sv1.data, sv2.data))
aligned = np.exp(-1j*phase_est) * sv2.data

print("Statevectors identical after removing global phase:", np.allclose(sv1.data, aligned))


Probabilities equal: True
Probs: [0.5 0.5]
Statevectors identical after removing global phase: True


## Sample Questions (Multiple Choice)

**Q1.** What is the probability of measuring 0 in the state |+⟩ = (1/√2)(|0⟩ + |1⟩)?
- A. 1
- B. 0.5 
- C. 0
- D. 0.25

**Q2.** Which of the following is NOT a valid quantum state?
- A. [1/√2, 1/√2]
- B. [0.6, 0.8]
- C. [2, 2] 
- D. [1, 0]

**Q3.** In Dirac notation, which of the following represents the state vector [0, 1]?
- A. |0⟩
- B. |1⟩ 
- C. |+⟩
- D. |−⟩

Answers:
1: B, 2: C, 3: B